In [1]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Merge
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.
/Users/mich/anaconda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#df_train = pd.read_pickle("./data/df_train.pkl")
#df_test = pd.read_pickle("./data/df_test.pkl")

token_train = pd.read_pickle("./data/token_train.pkl")
token_test = pd.read_pickle("./data/token_test.pkl")

#vec_train = pd.read_pickle("./data/vec_train_glove.pkl")
#vec_test = pd.read_pickle("./data/vec_test_glove.pkl")

In [3]:
token_train.head()

,id,qid1,tokenq1,qid2,tokenq2,is_duplicate
0,0,0,"[what, is, the, step, by, step, guide, to, inv...",241804,"[what, is, the, step, by, step, guide, to, inv...",0
1,1,1,"[what, is, the, story, of, kohinoor, kohinoor,...",13483,"[what, would, happen, if, the, indian, governm...",0
2,2,2,"[how, can, i, increase, the, speed, of, my, in...",241805,"[how, can, internet, speed, be, increased, by,...",0
3,3,3,"[why, am, i, mentally, very, lonely, how, can,...",241806,"[find, the, remainder, when, math, two, three,...",0
4,4,4,"[which, one, dissolve, in, water, quikly, suga...",241807,"[which, fish, would, survive, in, salt, water]",0


# Word2Vec Embedding 

In [4]:
googlenews_embedding = './data/GoogleNews-vectors-negative300.bin.gz'
word2vec = KeyedVectors.load_word2vec_format(googlenews_embedding, binary=True)

vocabulary = dict()
inverse_vocabulary = ['<unk>']

In [5]:
stops = set(stopwords.words('english'))
questions = ['tokenq1', 'tokenq2']

# Iterate over the questions of both training and test datasets
for dataset in [token_train, token_test]:
    for index, row in dataset.iterrows():

        # Iterate through the text of both questions of the row
        for question in questions:

            question_to_number = []  # q2n -> question numbers representation
            for word in row[question]:

                # Check for unwanted words
                if word in stops and word not in word2vec.vocab:
                    continue

                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    question_to_number.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                else:
                    question_to_number.append(vocabulary[word])

            # Replace questions as word to question as number representation
            dataset.set_value(index, question, question_to_number)

/Users/mich/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [6]:
token_train.head()

,id,qid1,tokenq1,qid2,tokenq2,is_duplicate
0,0,0,"[1, 2, 3, 4, 5, 4, 6, 7, 8, 9, 10, 8, 11]",241804,"[1, 2, 3, 4, 5, 4, 6, 7, 8, 9, 10]",0
1,1,1,"[1, 2, 3, 12, 13, 13, 14]",13483,"[1, 15, 16, 17, 3, 18, 19, 20, 3, 13, 13, 14, 21]",0
2,2,2,"[22, 23, 24, 25, 3, 26, 27, 28, 29, 30, 31, 32]",241805,"[22, 23, 28, 26, 33, 34, 5, 35, 36, 37]",0
3,3,3,"[38, 39, 24, 40, 41, 42, 22, 23, 24, 43, 44]",241806,"[45, 3, 46, 47, 48, 49, 50, 49, 51, 48, 2, 52,...",0
4,4,4,"[53, 54, 55, 8, 56, 57, 58, 59, 60, 61, 62, 63]",241807,"[53, 64, 15, 65, 8, 59, 56]",0


In [7]:
# Build the embedding matrix
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
embeddings[0] = 0  # So that the padding will be ignored

for word, index in vocabulary.items():
    if word in word2vec.vocab:
        embeddings[index] = word2vec.word_vec(word)

del word2vec

# Prepare training and validation data

In [8]:
# split to train validation
validation_size = 40000
training_size = len(token_train) - validation_size

question_cols = token_train[['tokenq1', 'tokenq2']]
duplicate = token_train['is_duplicate']

X_train, X_validation, Y_train, Y_validation = train_test_split(question_cols, duplicate, test_size=validation_size)

# Split to dicts
X_train = {'left': X_train.tokenq1, 'right': X_train.tokenq2}
X_validation = {'left': X_validation.tokenq1, 'right': X_validation.tokenq2}
X_test = {'left': token_test.tokenq1, 'right': token_test.tokenq1}

In [9]:
# Convert labels to their numpy representations
Y_train = Y_train.values
Y_validation = Y_validation.values

In [17]:
X_train

{'left': 8199      [47, 269, 4349, 6503, 33, 3103, 108, 3, 3045, ...
 190158    [1, 2, 3, 11927, 2616, 38, 2, 123, 2459, 8, 26...
 31079        [373, 2, 3139, 3, 1593, 1594, 1184, 286, 1075]
 212430         [22, 94, 24, 2312, 3, 3147, 615, 7001, 6116]
 123395                    [38, 2, 169, 167, 215, 754, 1184]
 55780                       [1, 115, 116, 8100, 1231, 1658]
 44940     [1, 2, 3, 699, 3089, 108, 31, 1924, 8755, 1983...
 133108                         [22, 115, 2406, 1313, 8, 11]
 295125                       [230, 3, 180, 1812, 8, 3, 689]
 129193                              [1, 115, 3, 2103, 3179]
 251345                              [22, 81, 177, 976, 316]
 64872                  [1, 2, 3, 180, 219, 1361, 159, 4084]
 143778               [2, 3, 58488, 6558, 574, 3, 95, 24946]
 282046                   [22, 23, 24, 5515, 675, 175, 4055]
 43502     [22, 1107, 139, 8025, 17394, 115, 1001, 250, 4...
 120147    [38, 94, 1990, 8, 3, 147, 36512, 97, 13949, 19...
 11715          

In [21]:
#max_seq_length = max(token_train.tokenq1.map(lambda x: len(x)).max(),
#                     token_train.tokenq2.map(lambda x: len(x)).max(),
#                     token_test.tokenq1.map(lambda x: len(x)).max(),
#                     token_test.tokenq2.map(lambda x: len(x)).max())
max_seq_length = 30

In [22]:
# Zero padding
for dataset, side in itertools.product([X_train, X_validation], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)

In [24]:
X_train

{'left': array([[   0,    0,    0, ...,  628,   49,   51],
        [   0,    0,    0, ..., 2623, 2616,  449],
        [   0,    0,    0, ..., 1184,  286, 1075],
        ..., 
        [   0,    0,    0, ..., 9726,   80,  857],
        [   0,    0,    0, ...,  175,  123, 4007],
        [   0,    0,    0, ..., 8039,  602, 1067]], dtype=int32),
 'right': array([[    0,     0,   732, ...,   466,  3629, 13625],
        [    0,     0,     0, ...,  2616,  2623,    38],
        [    0,     0,     0, ...,  1184,   286,  1075],
        ..., 
        [    0,     0,     0, ...,  9726,    27,   857],
        [    0,     0,     0, ...,   424,   123,  4007],
        [    0,     0,     0, ...,  1067,     8,  6954]], dtype=int32)}

In [25]:
# TEST
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)

# Building the model

In [28]:
#n_hidden = 50
n_hidden = 1
gradient_clipping_norm = 1.25
#batch_size = 64
batch_size = 1
#n_epoch = 25
n_epoch = 10

In [29]:
# MaLSTM similarity function
def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

In [30]:
# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

In [31]:
# Calculates the distance as defined by the MaLSTM model
malstm_distance = Merge(mode=lambda x: exponent_neg_manhattan_distance(x[0], x[1]), output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

/Users/mich/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  


In [32]:
# Pack it all up into a model
malstm = Model([left_input, right_input], [malstm_distance])

In [33]:
# Adadelta optimizer, with gradient clipping by norm
optimizer = Adadelta(clipnorm=gradient_clipping_norm)

malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
# Start training
training_start_time = time()

malstm_trained = malstm.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, epochs=n_epoch,
                            validation_data=([X_validation['left'], X_validation['right']], Y_validation), validation_split = 0.2)

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

# Plotting results

In [ ]:
# Plot accuracy
plt.plot(malstm_trained.history['acc'])
plt.plot(malstm_trained.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot loss
plt.plot(malstm_trained.history['loss'])
plt.plot(malstm_trained.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()